In [1]:
# First, import the packages we'll need.
# From Rigetti:
from pyquil import get_qc, Program
from pyquil.gates import H, CNOT, PHASE, X, RX, MEASURE, RESET

# Other useful things
import numpy as np
import matplotlib.pyplot as plt
import itertools
import time
from typing import List

In [ ]:
# Declare a device and the qubits to use
qpu = get_qc("9q-generic-qvm", as_qvm=True, noisy=False)
qubits = qpu.device.qubits()[0:2] # We only need two qubits
# qpu = get_qc("Aspen-4-2Q-A")
# qubits = qpu.device.qubits()[16:17] # We only need two qubits

In [ ]:
# Define the function to iterate
def prepare_bell_singlet(qubits: List[int]) -> Program:
    program = Program()
    # put the first qubit into |+> state
    program.inst(H(qubits[0]))
    # Entangle the other qubit
    program.inst(CNOT(qubits[0], qubits[1]))
    # Flip the first qubit 
    program.inst(X(qubits[0]))
    # Add a phase to the second qubit
    program.inst(PHASE(np.pi,qubits[1]))
    return program

In [ ]:
program = Program()
ro = program.declare('ro', 'BIT', len(qubits))
program.inst(prepare_bell_singlet(qubits))
program.inst([MEASURE(qubit, ro[idx]) for idx, qubit in enumerate(qubits)])
program.wrap_in_numshots_loop(10_000)
binary = qpu.compile(program)

In [ ]:
# Set up a timer and run the program on the qvm
start = time.time()
results = qpu.run(binary)
total = time.time() - start
print(f'Execution time: {total:.3f} s')

# Examine the output
counts = {bit_tuple: 0 for bit_tuple in itertools.product((0, 1), repeat=len(qubits))}
for shot_result in results:
    bit_tuple = tuple(shot_result)
    counts[bit_tuple] += 1  
print(f'Measurement results:')
for bit_tuple, count in counts.items():
    print(bit_tuple, count)

In [ ]:
program = Program()
theta = program.declare('theta', 'REAL', 1)
phi = program.declare('phi', 'REAL', 1)
ro = program.declare('ro', 'BIT', 2)

program.inst(prepare_bell_singlet(qubits))
program.inst(RX(phi, qubits[0]))
program.inst(RX(theta, qubits[1]))
program.inst([MEASURE(qubit, ro[idx]) for idx, qubit in enumerate(qubits)])
program.wrap_in_numshots_loop(5000)

binary = qpu.compile(program)

In [ ]:
a  = np.pi
a_ = 3*np.pi/2
b  = np.pi/4
b_ = 3*np.pi/4

B_ = np.zeros([4,1])
D_ = np.zeros([4,1])

bell_results = qpu.run(binary,memory_map={'theta': [a],'phi':[b]})
B_[0] = np.mean(np.prod(2*bell_results-1,axis=1))
D_[0] = np.std(np.prod(2*bell_results-1,axis=1))
bell_results = qpu.run(binary,memory_map={'theta': [a],'phi':[b_]})
B_[1] = np.mean(np.prod(2*bell_results-1,axis=1))
D_[1] = np.std(np.prod(2*bell_results-1,axis=1))
bell_results = qpu.run(binary,memory_map={'theta': [a_],'phi':[b]})
B_[2] = np.mean(np.prod(2*bell_results-1,axis=1))
D_[2] = np.std(np.prod(2*bell_results-1,axis=1))
bell_results = qpu.run(binary,memory_map={'theta': [a_],'phi':[b_]})
B_[3] = np.mean(np.prod(2*bell_results-1,axis=1))
D_[3] = np.std(np.prod(2*bell_results-1,axis=1))
CHSH_value=B_[0]-B_[1]+B_[2]+B_[3]
CHSH_uncert = sum(D_)/np.sqrt(4*5000)
excess = abs(CHSH_value)-2
sigma = excess/CHSH_uncert
if excess > 0:
    print('You violated the Bell inequality by %.3f with uncertainty sigma=%.3f !'%(excess,sigma))
    print('plz nobel prize thx')
else:
    print('You call that entangled??')

In [ ]:
!pip install pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
Paulis = np.zeros((3, 2, 2), dtype=complex)
Paulis[0] = [[0, 1], [1, 0]]
Paulis[1] = [[0, -1j], [1j, 0]]
Paulis[2] = [[1, 0], [0, -1]]
Correlator = np.kron(Paulis[2],Paulis[2])